In [ ]:
%matplotlib inline

import sys
from pathlib import Path

from tqdm.notebook import tqdm
import numpy as np
import pandas as pd

from bananas.utils import images
from bananas.utils.arrays import unique
from bananas.dataset import DataSet, DataType, Feature

# Root path of project relative to this notebook
ROOT = Path('..')

sys.path.insert(1, str(ROOT / 'scripts'))
from datamodels import *
from utils import *

### Read patient data from local file

In [ ]:
df = pd.read_csv(ROOT / 'datasets' / 'subject_diagnosis.csv', index_col=0)

# Convert non-primitive fields
df['processed_path'] = df['processed_path'].apply(lambda x: Path(x))
df['image_path'] = df['image_path'].apply(lambda x: Path(x))
df['template_path'] = df['template_path'].apply(lambda x: Path(x))
df['drawing_box'] = df['drawing_box'].apply(lambda x: Box.load(x))
df['template_box'] = df['template_box'].apply(lambda x: Box.load(x))

# Remove all unnecessary columns from our dataset
feat_keys = ['processed_path']
group_columns = ['diagnosis']
df = df[group_columns + feat_keys].copy()

# Normalize all feature columns
df = df.dropna()
for col in feat_keys:
    df[col] = df[col].apply(lambda x: str(ROOT / x))

df.head()

### Load custom MLP model

In [ ]:
from quick_draw_learner import QDClassifier

### Build and train...

In [ ]:
from itertools import product, combinations
    
# Define all possible hyperparameters
kernel_sizes = [3, 5, 7, 9]
batch_sizes = [24, 32, 48]
test_splits = [.2, .25]
validation_splits = [.2, .25]

# Initialize random number generator without seed to randomize hyperparamters
rnd = np.random.RandomState()

# Cross product all hyperparameters
parameter_combinations = list(product(
    kernel_sizes, batch_sizes, test_splits, validation_splits))
rnd.shuffle(parameter_combinations)

target_label = 'SANO'
target_column = 'diagnosis'

In [ ]:
from bananas.sampling.cross_validation import DataSplit
from bananas.statistics import scoring
from bananas.statistics.scoring import ScoringFunction

# Store results in a list to display them later
trial_results = []

# Perform 3 trials per parameter set to later compute the average
trial_count = 3

for kernel_size, batch_size, test_split, validation_split in tqdm(parameter_combinations, leave=False):

    # Re-initialize seed every time
    random_seed = 0
    
    # Execute the independent trials
    for _ in tqdm(range(trial_count), leave=False, desc='Trial'):

        # Create a single feature containing all image data
        features = [Feature(
            ImageAugmenterLoader(df['processed_path'].values),
            kind=DataType.HIGH_DIMENSIOAL,
            sample_size=10,
            random_seed=random_seed)]

        # Define target feature
        target_feature = Feature(
            (df[target_column] == target_label).values, random_seed=random_seed)

        while True:

            # Build dataset, making sure that we have a left-out validation subset
            dataset = DataSet(
                features,
                name=target_label,
                target=target_feature,
                random_seed=random_seed,
                batch_size=batch_size,
                test_split=test_split,
                validation_split=validation_split)

            # Compute test class balance to tell what minimum accuracy we should beat
            test_idx = dataset.sampler.subsamplers[DataSplit.VALIDATION].data
            test_classes = target_feature[test_idx]
            test_class_balance = sum(test_classes) / len(test_classes)

            # Rebuild dataset unless test class balance is within 5% of ground truth
            true_class_balance = sum(target_feature[:] / len(target_feature))
            if abs(test_class_balance - true_class_balance) < .05: break

            # Keep changing the seed to avoid getting stuck
            random_seed += 1

        # Instantiate learner
        learner = QDClassifier(
            kernel_size=kernel_size,
            random_seed=random_seed, 
            verbose=False)

        # Train learner using train dataset
        learner.train(dataset.input_fn, progress=True, max_steps=1000)

        # Test learner predictions using left-out dataset
        X, y = dataset[test_idx]
        y = learner.label_encoder_.transform(y)
        y_ = learner.predict_proba(X)
        score_auroc = scoring.score_auroc(y, y_)
        score_accuracy = scoring.score_accuracy(y, y_)

        # Store trial results
        naive_accuracy = max(test_class_balance, 1 - test_class_balance)
        trial_results.append({
            'Subset Splits': (test_split, validation_split),
            'Kernel size': kernel_size,
            'Batch size': batch_size,
            'Δ Naive Classifier': score_accuracy - naive_accuracy,
            'Accuracy': score_accuracy,
            'Area under ROC': score_auroc,
        })
        
        # Change seed to ensure different dataset split
        random_seed += 1

In [ ]:
res = pd.DataFrame.from_records(trial_results).sort_values('Accuracy', ascending=False)
res.to_csv('qd_ungrouped_results.csv')
res.head(15)